On reloade tous les objets précédents

In [1]:
import pandas as pd
import pickle
import time

path = '/kaggle/input/pawpularity-features-model/'
debut = time.time()
df = pickle.load(open(path + 'X_features_cleaned_2.pkl', 'rb'))
correlated_features = pickle.load(open(path + 'correlated_features.pkl', 'rb'))
correlated_features_2 = pickle.load(open(path + 'correlated_features_2.pkl', 'rb'))
print("Temps écoulé : {:.2f}".format(time.time() - debut))

In [2]:
df.shape

In [3]:
y_path = "/kaggle/input/pawpularity-features-targets/y_targets.csv"
df_target = pd.read_csv(y_path, header=None)
df_target.shape

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, df_target, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Test avec XGBoost, CatBoost & LightGBM

In [5]:
from lightgbm import LGBMRegressor
lgbm_regressor = LGBMRegressor()
lgbm_regressor.fit(X_train, y_train)

In [6]:
y_pred = lgbm_regressor.predict(X_test)

In [7]:
print(y_pred.shape, y_test.values.shape)
ypred = [round(x) for x in y_pred]
ytest = y_test[0].values

In [8]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

print("mean_absolute_error : {:.2f}".format(mean_absolute_error(ytest, ypred)))
mse = mean_squared_error(ytest, ypred)
print("mean_squared_error : {:.2f}".format(mse))
print("root_mean_squared_error : {:.2f}".format(np.sqrt(mse)))
print("rmpse : {:.2f}".format(np.sqrt(np.mean(np.square(((ytest - ypred) / ytest))))))
print("r2_score : {:.2f}".format(r2_score(ytest, ypred)))

In [29]:
import os
from PIL import Image

test_dir = '/kaggle/input/petfinder-pawpularity-score/test'

test_df = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/test.csv", sep=',')
test_df['Id'] = test_df['Id'] + '.jpg'
test_df = test_df.rename(columns={'Id': 'filename'})
print("shape test_df: ", test_df.shape)

width, height = 224, 224
batch_size = 32

import sys
import os
sys.path.insert(0, "/kaggle/input/efnetv2src/efficientnet-v2-keras-main")
sys.path.append('../input/tfkeras-efficientnetsv2/')

from efficientnet_v2 import EfficientNetV2XL
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.metrics import MeanAbsoluteError, MeanAbsolutePercentageError
import tensorflow_addons as tfa

efficientnet = EfficientNetV2XL(
    include_top=False,
    weights='../input/tfkeras-efficientnetsv2/21_ft1k_notop/efficientnetv2-xl-21k-ft1k_notop.h5', 
    input_shape=(height, width, 3)
)

efficientnet.trainable = False

radam = tfa.optimizers.RectifiedAdam(learning_rate=0.001)
optimizer = tfa.optimizers.Lookahead(radam, sync_period=6, slow_step_size=0.5)

efficientnet.compile(
    optimizer=optimizer,
    loss="mean_absolute_error",
    metrics=[MeanAbsoluteError(), MeanAbsolutePercentageError()]
)

In [58]:
# on merge les 2 listes des index à supprimer
for x in correlated_features_2:
    correlated_features.append(x)

In [59]:
# on récupère le nombre de colonnes à ajouter à la sortie d'EfficientNet
existing_cols = test_df.iloc[0]
existing_cols = existing_cols.drop(labels=['filename']).values
nb_existing_cols = len(existing_cols)

# on décale les index des colonnes à supprimer
print(correlated_features[:5])
correlated_features = [x - nb_existing_cols for x in correlated_features]
print(correlated_features[:5])

In [82]:
import tensorflow as tf
def preprocess(image):  
    return (tf.cast(image, dtype=tf.float32) - 128.00) / 128.00

ids = []
pawpularities = []
for test_image in os.listdir(test_dir):
    image_path = os.path.join(test_dir, test_image)
    id_image = test_image.split('.')[0]
    ids.append(id_image)
    img = Image.open(image_path) 
    img = img.resize((width, height))
    img = preprocess(np.array(img).reshape(1, width, height, 3))
    # on fait la prediction avec EfficientNet
    img_features = efficientnet.predict(img)
    img_features = img_features.flatten()
    # on supprime les colonnes corrélées
    img_features = np.delete(img_features, correlated_features)
    # on récupère les colonnes existantes
    existing_cols = test_df[test_df['filename'] == test_image].iloc[0]
    existing_cols = existing_cols.drop(labels=['filename']).values
    # on concatène tout ça et on reshape (1 seule ligne)
    X_pred = np.concatenate((np.array(existing_cols), img_features))
    X_pred = X_pred.reshape((1, len(X_pred)))
    # on fait la prediction finale
    y_pred = xgb_regressor.predict(X_pred)
    # on ajoute ça à la liste des prédictions
    pawpularities.append(round(y_pred[0]))

submission_dict = {
    'Id': ids,
    'Pawpularity': pawpularities
}
submission_df = pd.DataFrame(submission_dict)
print(submission_df)
submission_df.to_csv('submission.csv', index=False, sep=',')